In [92]:
import tensorflow as tf
import pandas as pd
from sklearn import *
import numpy as np

In [2]:
tf.__version__

'1.4.0'

In [119]:
df = pd.read_csv("/data/Combined_Cycle_Power_Plant.csv")
df.head()
y = df.PE.values.reshape(-1, 1)
X = df.iloc[:, :-1].values.astype(np.float32)
X = preprocessing.StandardScaler().fit_transform(X)
print(X.shape, y.shape)
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.3, random_state = 134)

(47840, 4) (47840, 1)


In [120]:
lr = linear_model.LinearRegression()
lr.fit(X_train, y_train)
print("MSE: ", metrics.mean_squared_error(y_test, lr.predict(X_test)))
lr.intercept_, lr.coef_

MSE:  20.8099125274


(array([ 454.35983276], dtype=float32),
 array([[-14.69581604,  -3.0020566 ,   0.38572252,  -2.29661846]], dtype=float32))

In [135]:
tf.reset_default_graph()
n_x, n_y = 4, 1 # number of features of df is 4, and output is single value 
max_iter = 10000

X = tf.placeholder(dtype=tf.float32, shape=[None, n_x], name = "X")
y = tf.placeholder(dtype=tf.float32, shape=[None, n_y], name = "y")

print("X:", X)
print("y:", y)

#W = tf.Variable(initial_value = tf.zeros([n_x, n_y], dtype=tf.float32), name="W")
W = tf.Variable(tf.truncated_normal([n_x, n_y], stddev=0.1, seed = 1, dtype=tf.float32), name = "W")

print("W:", W)

b = tf.Variable(0.0, dtype=tf.float32, name="b")

Z = tf.matmul(X, W) + b
print("Z: ", Z)

y_pred = tf.nn.relu(Z)

residual = (y_pred - y)
print("residual", residual)

mse = tf.reduce_mean(tf.square(residual))
print("MSE", mse)

cost = mse

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(max_iter):
        cost_, _ = sess.run([cost, optimizer], feed_dict={X: X_train, y: y_train})
        if i % (max_iter//10) ==0:
            print(cost_)
    
    mse_ = sess.run(mse, feed_dict={X: X_test, y: y_test})
    print("Test MSE", mse_)

X: Tensor("X:0", shape=(?, 4), dtype=float32)
y: Tensor("y:0", shape=(?, 1), dtype=float32)
W: <tf.Variable 'W:0' shape=(4, 1) dtype=float32_ref>
Z:  Tensor("add:0", shape=(?, 1), dtype=float32)
residual Tensor("sub:0", shape=(?, 1), dtype=float32)
MSE Tensor("Mean:0", shape=(), dtype=float32)
206616.0
20.7549
20.7549
20.7549
20.7549
20.7549
20.7549
20.7549
20.7549
20.7549
Test MSE 20.8099


In [122]:
tf.zeros(shape= [n_x, n_y])

<tf.Tensor 'zeros:0' shape=(4, 1) dtype=float32>

In [79]:
tf.zeros?